In [ ]:
import PyPDF2
import requests
from urllib.request import Request, urlopen
import io
import pandas as pd
import re
import numpy as np

Colleges: <br>
AG: Agriculture and Life Sciences<br>
AR: Architecture<br>
GB: Bush School of Government and Public Service<br>
BA: Business<br>
DN: Dentistry<br>
ED: Education<br>
EN: Engineering<br>
GE: Geosciences<br>
LA: Liberal Arts<br>
MS: Military Science<br>
NU: Nursing<br>
PH: Public Health<br>
SC: Science<br>
VM: Vet Med<br>
Terms:<br>
1: Spring<br>
2: Summer<br>
3: Fall<br>
<br>
Data available from Fall 2017 - Spring 2022 as of Sept 9, 2022

In [ ]:
colleges = ['AG', 'AR', 'GB', 'BA', 'DN','ED','EN','GE', 'LA',
            'MS','NU','PH','SC','VM']
terms = ['1','2','3'] 
years = ['2018','2019','2020','2021']

urls = []
#Only Fall available in 2017 
for college in colleges:
    urls.append('https://web-as.tamu.edu/GradeReports/PDFReports/20173/grd20173' + college + '.pdf')

#Get most of course data
for college in colleges:
    for year in years:
        for term in terms:
            urls.append('https://web-as.tamu.edu/GradeReports/PDFReports/'+ year + term + '/grd' + year + term + college + '.pdf')

#Only Spring available in 2022           
for college in colleges:
    if college == 'DN':
        continue
    urls.append('https://web-as.tamu.edu/GradeReports/PDFReports/20221/grd20221' + college + '.pdf')


In [ ]:
rows = []
for URL in urls:
    print(URL)
    #Find file from URL
    file = urlopen(Request(URL)).read()
    mem_file = io.BytesIO(file)
    fileReader = PyPDF2.PdfFileReader(mem_file)
    
    #Find Course Name to parse the PDF
    course_list = set()
    page_count = fileReader.numPages
    for page_num in range(page_count):
        current_page = fileReader.getPage(page_num).extractText()
        courses = re.findall("[A-Z][A-Z][A-Z][A-Z]-[0-9]", current_page) 
        courses = set(courses)
        courses = list(courses)
        for elem in courses:
            course_list.add(elem)
            
    #Parse through PDF and create rows for data frame
    for page_num in range(page_count):
        current_page = fileReader.getPage(page_num).extractText().split()
        for num in range(len(current_page)):
            current_row = []
            
            #If the first 6 characters of the row is a valid course, create a row of data
            if current_page[num][:6] in course_list:
                instructor_name = ''
                try:
                    instructor_name = current_page[num+19] + ' ' + current_page[num+20]
                except:
                    instructor_name = current_page[num+19]
                current_row = current_page[num:num+19]
                current_row.append(instructor_name)
                
                #Add Year, Term, and College to record
                current_row.append(URL[48:52])
                current_row.append(URL[52])
                current_row.append(URL[62:64])
                #Add record to nested list
                rows.append(current_row)       

In [ ]:
col_names = ['Course_Info', 'A', 'A percentage', 'B', 'B percentage',  'C', 'C percentage',
             'D', 'D percentage', 'F', 'F percentage', 'Total Completed', 'GPA',
             'I','S','U','Q','X', 'Total Registered', 'Instructor', 'Year','Term','College']
df = pd.DataFrame(rows, columns = col_names)

In [ ]:
df.head()

In [ ]:
#Divide up Course Info for more convenient analysis in R
df['Course'] =  df['Course_Info'].str[:8]
df['Course_Num'] =  df['Course_Info'].str[5:8]
df['Dept'] = df['Course_Info'].str[:4]
df['Section'] = df['Course_Info'].str[9:12]
df['TermYear'] = df['Year'] + ' ' + df['Term']

In [ ]:
conditions = [
    (df['Term'] == '1'),
    (df['Term'] == '2'),
    (df['Term'] == '3')    
]
choices = ['Spring', 'Summer', 'Fall']
df['Semester'] = np.select(conditions, choices)

In [ ]:
df.to_csv('tamu_grade_reports.csv', index = False)